In [0]:
def ingestion(df):
    df.createOrReplaceTempView('payload')
    
    spark.sql('''
        INSERT INTO silver.nytimes.top_stories_test
        REPLACE USING (ref_date)
        SELECT
        title AS ds_headline,
        abstract AS ds_lead,
        SPLIT(REPLACE(REPLACE(byline, 'By ', ''), ' and ', ', '), ', ') AS ds_authors,
        section AS ds_section,
        subsection AS ds_subsection,
        url AS ds_url,
        geo_facet AS ds_locations,
        des_facet AS ds_topics,
        org_facet AS ds_organizations,
        per_facet AS ds_persons,
        published_date,
        ref_date AS ref_date
        FROM payload        
    ''')

    return df

In [0]:
df_stream = (spark.readStream
    .option('readChangeFeed', 'true')
    .table('bronze.nytimes.top_stories')
)

In [0]:
stream = (df_stream.writeStream
    .option('checkpointLocation', '/Volumes/silver/nytimes/top_stories_checkpoint_test')
    .foreachBatch(lambda batch_df, batch_id: ingestion(batch_df))
    .trigger(availableNow=True)
    #.toTable('silver.nytimes.top_stories_test')          
)

In [0]:
stream.start()

In [0]:
%sql

SELECT * FROM silver.nytimes.top_stories_test;